In [1]:
%pip install accelerate
%pip install sacrebleu

import torch
from transformers import pipeline
from tqdm import tqdm
import json
from sacrebleu.metrics import BLEU



# save the challenge set in json format as lists

references = []
sources = []

with open("Challenge_set-v2hA.json", 'r', encoding="utf-8") as f:
    for line in f:
        data = json.loads(line)
        references.append(data["reference"])
        sources.append(data["source"])

# remove all the punctuations from the references and sources

import string

def remove_punctuations(text):
    return text.translate(str.maketrans('', '', string.punctuation))

references_cleaned = [remove_punctuations(ref) for ref in references]
sources_cleaned = [remove_punctuations(src) for src in sources]

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
references_cleaned[:10]

['Les appels répétés de sa mère auraient dû nous alerter',
 'Le bruit soudain dans les chambres supérieures aurait dû nous alerter',
 'Leurs échecs répétés à signaler le problème auraient dû nous alerter',
 'Elle a demandé à son frère de ne pas se montrer arrogant',
 'Elle a promis à son frère de ne pas être arrogante',
 'Elle a promis à son médecin de demeurer active après sêtre retirée',
 'Ma mère a promis à mon père dêtre plus prudente sur la route',
 'La femme était très grande et extrêmement forte',
 'Leurs politiciens étaient plus ignorants que stupides',
 'Nous avons lancé une insulte et nous sommes partis brusquement']

In [ ]:
sources_cleaned[:10]

['The repeated calls from his mother should have alerted us',
 'The sudden noise in the upper rooms should have alerted us',
 'Their repeated failures to report the problem should have alerted us',
 'She asked her brother not to be arrogant',
 'She promised her brother not to be arrogant',
 'She promised her doctor to remain active after retiring',
 'My mother promised my father to be more prudent on the road',
 'The woman was very tall and extremely strong',
 'Their politicians were more ignorant than stupid',
 'We shouted an insult and left abruptly']

In [ ]:
# default translation model

translator = pipeline("translation_en_to_fr")

No model was supplied, defaulted to google-t5/t5-base and revision 686f1db (https://huggingface.co/google-t5/t5-base).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5_fast.py:171: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on google-t5/t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [ ]:
# translate the sources (in French) in English

translated_sources = [translator(src) for src in tqdm(sources_cleaned)]

100%|██████████| 108/108 [03:14<00:00,  1.80s/it]


In [ ]:
# keep only the tranlated sentences

translated_sources = [src[0]['translation_text'] for src in translated_sources]
translated_sources[:10]

['Les appels répétés de sa mère auraient dû nous avertir',
 'Le bruit soudain dans les chambres supérieures aurait dû nous alerter.',
 'Leur refus répété de signaler le problème aurait dû nous avertir',
 'Elle a demandé à son frère de ne pas être arrogant',
 'Elle a promis à son frère de ne pas être arrogante',
 'Elle a promis à son médecin de demeurer active après sa retraite',
 'Ma mère a promis à mon père d’être plus prudent sur la route.',
 'La femme était très grande et extrêmement forte',
 'Leurs politiques étaient plus ignorantes que stupides',
 'Nous avons crié une insulte et nous sommes brusquement partis.']

In [ ]:
# bleu score of the default model

bleu = BLEU()
bleu_score_llm = bleu.corpus_score(translated_sources, references_cleaned)
bleu_score_llm.score

0.44822138964642555